# Focal Loss

## Class Imbalance

간단하게 설명해서, 이미지 하나당 40000~50000개 정도를 추천하는데, 이중에 object가 있을거라 추정하는 위치는 매우 적습니다. <br>
문제는 다음과 같습니다.

* 대부분의 위치들은 **easy negatives** (background라고 판단되는 확률값이 0.5 이상인 녀석들)이며, easy negatives는 실제 학습에 크게 도움을 주지 않습니다. <br>easy negatives들 자체의 loss값은 작지만 이런 값들 자체의 양이 많고, 모두 합친 loss값은 rare class에서 나온 loss값을 압도해버립니다. 


> <span style="color:#999999"> 페이퍼에서 **easy** 라는 말이 많이 쓰이는데.. 해당 class (background 또는 foreground)를 0.5 확률이상으로 예측한 것을 말합니다. <br>
> 즉 높은 확률값으로 배경이다 또는 객체이다를 예측한 위치를 말합니다.</span>


## Cross Entropy

Focal Loss은 one-stage object detection 시나리오를 다루기 위해서 디자인 되었으며 학습도중의 foreground 그리고 background classes의 극단적인 class imbalance 문제를 해결합니다.

먼저 binary cross entropy를 시작으로 해서 focal loss를 정의하도록 합니다.

$$ \text{CE}(p, y) = 
\begin{cases} 
- \log(p) & \text{if}\ y = 1 \\ 
- \log(1-p) & \text{otherwise} \\
\end{cases} $$

* $ y \in \{ \pm 1 \} $  :  y 는  1 또는 -1 이며, ground-truth class를 나타낸다.
* $ p \in [ 0 , 1 ]  $ : p는 0 ~ 1 사이의 값이며, 모델이 추정한 확률값 (estimated probability) 입니다 (1에 가까울수록 ground-truth class)

notation을 줄여서 쓰기 위해서 $ P_t $ 를 다음과 같이 정의 합니다.

$$ P_t = 
\begin{cases} 
p & \text{if } y = 1 \\
1 - p & \text{otherwise}
\end{cases} $$

$$ CE(p, y) = CE(p_t) = -log(p_t) $$

## Balanced Cross Entropy

class imbalance문제를 해결하는 가장 일반적인 방법은 weighting factor $ \alpha \in [0, 1] $ 를 class 1에 그리고 $ 1 - \alpha $ 를 class -1 에 적용하는 것 입니다.

$$ \text{CE}(p_t) = - \alpha_t \log(p_t) $$

## Focal Loss Definition

페이퍼의 연구 결과가 말해주듯이, 대부분의 loss값 그리고 대부분의 gradient값을 차지하는 것은 easily classified negatives입니다. <br>
$ \alpha $ 를 사용해서 positive/negative examples 의 중요도를 조정할 수 있지만, easy/hard examples은 구별하지 못합니다. 

Focal loss에서는 easy examples은 down-weight를 주고, hard examples에 집중할 수 있도록 해줍니다. <br>
구체적으로는 modulating factor $ (1 - p_t)^\gamma $ 를 cross entropy에 추가를 합니다. 이때 $ \gamma \ge 0 $ 입니다. 

$$ \text{FL}(p_t) = - (1-p_t)^\gamma \log(p_t) $$

아래는 $ \gamma \in [0, 5] $ 값에 따른 그래프의 변화입니다.

![Focal Loss](images/retina-focal-loss-graph.png)

<span style="color:red">
**쉽게 설명하면!** 예측한 값이 ground-truth class에 맞을수록 영향도를 적게 주고, <br>
object를 background라고 하거나 background 를 object라고 한것처럼 개틀린것은 포커스하겠다는 뜻입니다.</span>

아래 코드를 보면.. P값이 높을수록 ground-truth class에 맞는건데, factor값이 0에 가깝습니다.<br>
예를 들어서 P값이 0.99 는 factor 값이 0.000001이고, P값이 0.01은 factor값이 0.97 입니다.

> 논문에서는 $ \gamma = 2 $ 일때 가장 학습이 잘 되었다고 합니다.

In [107]:
import numpy as np
import pandas as pd

def focal_loss(y_true, y_pred, gamma=1, visual=True):
    p = (y_true * y_pred) + ((1 - y_true) * (1 - y_pred))
    factor = (1 - p)**gamma
    loss = - (factor * np.log(factor))
    
    # Visualize    
    if visual:
        df = pd.DataFrame({'y_true': y_true,
                           'y_pred': y_pred,
                           'p': p,
                           'factor': factor,
                           'loss': loss}, columns=['y_true', 'y_pred', 'p', 'factor', 'loss'])
        display(df)
    return loss.mean()

y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1], dtype=np.float64)
y_pred = np.array([0.99, 0.6, 0.3, 0.01, 0.01, 0.3, 0.6, 0.99], dtype=np.float64)
print('loss:', focal_loss(y_true, y_pred, 3))

,y_true,y_pred,p,factor,loss
0,0.0,0.99,0.01,0.970299,0.029255
1,0.0,0.60,0.40,0.216000,0.331015
2,0.0,0.30,0.70,0.027000,0.097522
3,0.0,0.01,0.99,0.000001,0.000014
4,1.0,0.01,0.01,0.970299,0.029255
5,1.0,0.30,0.30,0.343000,0.367019
6,1.0,0.60,0.60,0.064000,0.175928
7,1.0,0.99,0.99,0.000001,0.000014


loss: 0.12875271939682145


## Focal Loss with Alpha-balanced variant

<span style="color:red">**결론적으로 아래의 loss function을 실제 구현시에 사용합니다. **</span><br>
실제 focal loss를 적용할때는 $ \alpha $-balanced variant 를 함께 넣습니다. 

$$ \text{FL}(p_t) = - \alpha_t (1-p_t)^\gamma \log(p_t) $$

또한 실제 구현시에는 $ p $ 값에 sigmoid operation을 적용해줍니다. 

# Feature Pyramid Networks

* [Understanding Feature Pyramid Networks for object detection (FPN)](https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c) 에서 내용을 참고 하였습니다. 


Feature pyramids는 서로 다른 크기의 객체를 인식하는데 사용되는 방법입니다. 다만 잘 사용되지 않았는데.. computation그리고 메모리가 너무 많이 사용되기 때문이기 때문입니다. 

![Feature Pyramids](images/feature-pyramids.png)

* **(a) Featurized image pyramid:** 과거에 사용한 방법으로, 이미지 자체를 피라미드화해서 각각 연산을 합니다. 물론 예측률이 높지만 연산비용이 매우 높습니다. 
* **(b) Single feature map:** 빠른 연산이 장점이나, 축소된 정보에서 feature를 꺼낼때 많은 정보 손실이 일어납니다. 즉 예측률이 낮아집니다. 
* **(c) Pyramidal feature hierarchy:** ConvNet에서 처리된 각각의 features들을 사용하는 방법입니다. 문제는 ConvNet의 특징이 처음 이미지에서 feature를 뽑아낼때 점, 선들이다가 딥하게 내려갈수록 얼굴, 자동차같은 형태의 feature들이 뽑히게 됩니다. 즉 그림에서 아래쪽 features들일수록 실제 의미있는 feature들이 없다는게 문제 입니다.
* **(d) Feature Pyramid Network (FPN):** b와 c처럼 빠르지만 더 정확합니다. upsampling을 사용하여 마지막 단계의 features들을 살리면서, 처음단계의 features들까지 사용하게 됩니다.

(d) Feature Pyramid Network 의 경우 top-down pathway를 통해서 semantic 정보를 high resolution으로 reconstruction하게 됩니다. 문제는 downsampling  그리고 upsampling에서 나오는 정확한 위치 정보에 대한 손실이 일어나게 되는데 보완해 주는것이 일치하는 feature maps과 reconstructed layers를 skip connection처럼 연결해주는 것입니다. 

## Semantic Value VS Resolution

Resolution과 semantic value와 반비례하는 관계라고 보면 됩니다. <br>
Bottom layers는 high resolution이지만 semantic value가 부족하여 객체 인식에 적합하지 못하고, <br>
upper layers의 경우는 semantic value가 높지만, low resolution이기 때문에 작은 객체를 못잡아 냅니다.

![Data Flow](images/fpn-semantic.jpeg)

## Image Pyramid VS Convolution Neural Network 

Image pyramid의 경우 이미지마다 다양한 크기의 features들을 뽑아낼수 있지만, 앞서 말했듯이 연산비용이 너무 높습니다. <br>
Deep ConvNet의 경우 feature hierarchy를 레이어마다 연산을 하고, sub-sampling layer마다 다른 크기의 features를 뽑아내기 때문에 이미 multi-scale부분이 내재화 되어 있다고 볼 수 있습니다. 다양한 크기의 feature maps을 뽑아낼수 있지만 있지만 문제는, high-resolution maps일수록 low-level features (점, 선같은것들)를 갖고 있으며 실질적으로 object recognition에 쓰이기에는 문제가 있습니다.

## Single Shot Detector (SSD)

SSD의 ConvNet을 features를 마치 featurized image pyramid처럼 사용하는 초기모델 중의 하나 입니다. <br>
하지만 low-level features을 피하기 위해서 SSD는 upper layers들만 사용합니다. <br>
때문에 여전히 작은 객체는 잡지를 못하고, 연산량은 많습니다.


![SSD](images/fpn-ssd.png)



## Bottom-up Pathway

Buttom-up pathway는 기존 ConvNet에서 생성되는 feature maps이라고 생각하면 됩니다. <br>
stride 값이 2이기 때문에 위로 올라갈때마다 spatial 정보를 1/2 잃게 됩니다.

![Bottom Up](images/fpn-buttom-up.png)

## Top-down Pathway

Top-down pathway은 higher pyramid levels에서 upsampling을 함으로서 spatial information 부족하지만 semantic information은 강한 features 들을 사용함으로서 high resolution을 대략적으로 구현하게 됩니다.

C5 channel depth를 256-d 으로 줄이기 위해서 1 x 1 convolution을 적용하여 M5 가 나오게 됩니다.<br>
이후 3 x 3 convolution을 적용하여 P5를 생성하며, P5는 객체인식을 위한 첫번째 feature map이 되게 됩니다.

Top-down path를 내려가면서, 2만큼 nearest neighbors upsampling을 사용하여 upsampling을 하게 됩니다.<br>
다시 1 x 1 convolution을 상응하는 feature maps (C4)에 적용해준뒤 element-wise로 서로 더해주게 됩니다. <br>
그리고 3 x 3 convolution을 적용하여 두번째 feature maps을 만들게 됩니다. 

이런식으로 반복적으로 최종 feature maps을 생성하면 aliasing effect of upsampling효과를 차단할 수 있습니다.

## FPN with RPN

FPN자체는 object detector가 아니고, feature detector (또는 feature extractor)로서 object detector인 RPN과 연결이 됩니다. <br>
FPN에서 추출된 feature maps을 RPN의 3 x 3 convolution layer에 태운후 -> 다시 1 x 1 convolution을 통해서 object detecting class 그리고 bounding box regression을 하게 됩니다.

![FPN RPN](images/fpn-rpn.png)